### ETL
El proceso ETL es un paso crucial para preparar los datos para su análisis. Al **extraer, transformar y cargar** los datos de forma adecuada, se obtienen datos de mejor calidad y se facilita la obtención de información valiosa a partir de ellos.

### Importamos librerías

Comienza instalando solo las librerías que necesitas para empezar tu proyecto.

In [1]:
import pandas as pd
import pyarrow.parquet as pq
from textblob import TextBlob
import pyarrow as pa
from nltk.sentiment import SentimentIntensityAnalyzer

### La lectura de datos 

Lectura del archivo **australian_user_reviews.parquet**

In [2]:
#la siguiente línea de código lee el archivo australian_user_reviews.parquet y lo convierte en un DataFrame de Pandas
reviews = pq.read_table("../0-DATA/australian_user_reviews.parquet").to_pandas()

In [4]:
#presentación en un DataFrame los datos, analizamos como está estructurado nuestros datos.
reviews.head(3)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,None,"Posted July 15, 2011.",None,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,None,"Posted April 21, 2011.",None,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [7]:
#aplicamos el método .info() para ver la información a detalle estructuralmente de nuestros datos
reviews.info()# podemos visualizar que tenemos 59305  filas con 9 columnas, y cada columna tiene valores 'nulos'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        8151 non-null   object
 1   posted       59305 non-null  object
 2   last_edited  6140 non-null   object
 3   item_id      59305 non-null  int64 
 4   helpful      59305 non-null  object
 5   recommend    59305 non-null  bool  
 6   review       59275 non-null  object
 7   user_id      59305 non-null  object
 8   user_url     59305 non-null  object
dtypes: bool(1), int64(1), object(7)
memory usage: 3.7+ MB


### Transformación de nuestros Datos

Eliminación de valores duplicados

In [8]:
# analizaremos si tenemos filas duplicadas
duplicados = reviews.loc[reviews.duplicated()]#con un filtro visualizamos donde están solo las filas duplicadas
duplicados.shape #874 filas duplicadas

(874, 9)

In [9]:
#Eliminamos solo las filas duplicadas, el 1.41% del total de nuestra información
reviews = reviews.drop_duplicates(keep='first')
reviews.shape

(58431, 9)

Analizamos ahora si existen valores nullos

In [10]:
nulos = reviews.isnull().sum() # podemos ver que las columnas 'funny' y 'last_edited', casi en su totalidad son valores nulos
nulos

funny          50421
posted             0
last_edited    52394
item_id            0
helpful            0
recommend          0
review            30
user_id            0
user_url           0
dtype: int64

Eliminando valores nulos de la columna 'review'

In [11]:
# Eliminamos 30 filas, porcentaje ínfimo ni el 0.001%
reviews = reviews.dropna(subset=['review'])# que elimine solo valores nulos de la columna 'review'

Analizando columnas antes de eliminarlas

In [13]:
#antes de eliminar más filas analizaremos los detalles de nuestra data por columnas y precederemos según análisis
reviews["item_id"].unique() # no tiene valores nulos
#reviews["recommend"].unique() # no tiene valores nulos
#reviews["user_id"].unique() # no tiene valores nulos
#reviews["posted"].unique() # no tiene valores nulos
#reviews["review"].unique() # comentarios para el análisis de sentimiento
#reviews["funny"].unique() # --> eliminar ya que tiene 50421 nulos (87% del total de la data), además que tenemos la columna recommend que aporta mucho mas valor y sin nulos
#reviews["last_edited"].unique() # --> eliminar por la cantidad de nulos 52391 (90% del total)
#reviews["helpful"].unique() # --> eliminar, 51% no tiene rating, además que no es prescindible para nuestros endpoints
#reviews["user_url"].unique() # --> eliminar, estos links no es de importancia para nuestro modelo

array([  1250,  22200,  43110, ..., 220090, 262850, 431510], dtype=int64)

In [15]:
# Eliminando columnas
reviews = reviews.drop(columns=["funny", "last_edited", "helpful", "user_url"])
reviews.head(3)

,posted,item_id,recommend,review,user_id
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479


In [16]:
#verificamos si existen más valores nulos
nulos = reviews.isnull().sum() # visualizamos que no tenemos valores nulos.
nulos

posted       0
item_id      0
recommend    0
review       0
user_id      0
dtype: int64

### Analisis de Sentimiento

Cambiamos a minusculas los datos str de la columna 'review'

In [18]:
# convertimos a minúsculas para tener un óptimo análisis de sentimiento
reviews['review'] = reviews['review'].str.lower()
reviews.head(3) # visualizamos la columna 'review', ahora todo está en minúsculas

,posted,item_id,recommend,review,user_id
0,"Posted November 5, 2011.",1250,True,simple yet with great replayability. in my opi...,76561197970982479
1,"Posted July 15, 2011.",22200,True,it's unique and worth a playthrough.,76561197970982479
2,"Posted April 21, 2011.",43110,True,great atmosphere. the gunplay can be a bit chu...,76561197970982479


##### Analizaremos la columna **review** con dos librerías de análisis de sentimeinto

Modelo NLTK

In [19]:
def analizar_sentimiento_nltk(reviews):#Creamos una función para poner cada fila de la columna 'review' y procesar un análisis de sentimiento
    if reviews is None:
        return 1 # Si está vacía la celda o la revisión es nula, se clasifica como neutral

    # Analizamos el sentimiento con SentimentIntensityAnalyzer
    analyzer = SentimentIntensityAnalyzer()
    sentiment_score = analyzer.polarity_scores(reviews)['compound']

    # Ajustamos los umbrales para clasificar el sentimiento
    if sentiment_score > 0.4:
        return 2  # Positivo
    elif sentiment_score < -0.5:
        return 0  # Negativo
    else:
        return 1  # Neutral

In [20]:
# Aplicamos la función creada con el Modelo NLTK a la columna review y guardaremos en una nueva columna
reviews["sentiment_analysis"] = reviews["review"].astype(str).apply(analizar_sentimiento_nltk)
reviews.head(5)

,posted,item_id,recommend,review,user_id,sentiment_analysis
0,"Posted November 5, 2011.",1250,True,simple yet with great replayability. in my opi...,76561197970982479,2
1,"Posted July 15, 2011.",22200,True,it's unique and worth a playthrough.,76561197970982479,1
2,"Posted April 21, 2011.",43110,True,great atmosphere. the gunplay can be a bit chu...,76561197970982479,2
3,"Posted June 24, 2014.",251610,True,i know what you think when you see this title ...,js41637,2
4,"Posted September 8, 2013.",227300,True,for a simple (it's actually not all that simpl...,js41637,2


Modelo TextBlob

In [21]:
def analizar_sentimiento_textblob(review): #Creamos una función para poner cada fila de la columna 'review' y procesar un análisis de sentimiento
    if review is None:
        return 1  # Si está vacía la celda o la revisión es nula, se clasifica como neutral

    # Analizamos el sentimiento con TextBlob
    blob = TextBlob(review) # cada valor lo analizamos con 'TextBlob'
    sentiment_score = blob.sentiment.polarity # aquí damos un valor numérico segun lo procesado

    # Ajustamos los scores para clasificar el sentimiento
    if sentiment_score > 0.4: 
        return 2  # Positivo
    elif sentiment_score < -0.5:
        return 0  # Negativo
    else:
        return 1  # Neutral

In [22]:
# Aplicamos la función creada con el Modelo TextBlob a la columna review y guardaremos en una nueva columna
reviews["sentiment_analysis2"] = reviews["review"].astype(str).apply(analizar_sentimiento_textblob)
reviews.head(5)

,posted,item_id,recommend,review,user_id,sentiment_analysis,sentiment_analysis2
0,"Posted November 5, 2011.",1250,True,simple yet with great replayability. in my opi...,76561197970982479,2,1
1,"Posted July 15, 2011.",22200,True,it's unique and worth a playthrough.,76561197970982479,1,1
2,"Posted April 21, 2011.",43110,True,great atmosphere. the gunplay can be a bit chu...,76561197970982479,2,1
3,"Posted June 24, 2014.",251610,True,i know what you think when you see this title ...,js41637,2,1
4,"Posted September 8, 2013.",227300,True,for a simple (it's actually not all that simpl...,js41637,2,1


In [23]:
#análisis del resultado y que Modelo detecto más valores
NTK0 = len(reviews[reviews['sentiment_analysis']==0])
TB0 = len(reviews[reviews['sentiment_analysis2']==0])
NTK1 = len(reviews[reviews['sentiment_analysis']==1])
TB1 = len(reviews[reviews['sentiment_analysis2']==1])
NTK2 = len(reviews[reviews['sentiment_analysis']==2])
TB2 = len(reviews[reviews['sentiment_analysis2']==2])

print(f"comparamos los valores NLTK0 = {NTK0} vs TextBlob0 = {TB0}, NLTK1 = {NTK1} vs TextBlob1 = {TB1}, NLTK2 = {NTK2} vs TextBlob2 = {TB2} ")

comparamos los valores NLTK0 = 4883 vs TextBlob0 = 976, NLTK1 = 21835 vs TextBlob1 = 50317, NLTK2 = 31683 vs TextBlob2 = 7108 


NLTK tiene los mejores resultados, ya que analiza en más idiomas

In [24]:
# ya teniendo decidido los valores a trabajar del análisis de sentimiento, eliminamos la columna 'sentiment_analysis2'
reviews = reviews.drop('sentiment_analysis2', axis=1)
reviews.head(5)

,posted,item_id,recommend,review,user_id,sentiment_analysis
0,"Posted November 5, 2011.",1250,True,simple yet with great replayability. in my opi...,76561197970982479,2
1,"Posted July 15, 2011.",22200,True,it's unique and worth a playthrough.,76561197970982479,1
2,"Posted April 21, 2011.",43110,True,great atmosphere. the gunplay can be a bit chu...,76561197970982479,2
3,"Posted June 24, 2014.",251610,True,i know what you think when you see this title ...,js41637,2
4,"Posted September 8, 2013.",227300,True,for a simple (it's actually not all that simpl...,js41637,2


In [25]:
#visualizamos los valores únicos de la nueva columna
reviews.sentiment_analysis.unique()

array([2, 1, 0], dtype=int64)

Eliminamos la columna **review**

In [26]:
#el análisis de sentimiento cubre la informacion de la columna 'review', por tanto se elimina esta columna
reviews = reviews.drop('review', axis=1)
reviews.head(5)

,posted,item_id,recommend,user_id,sentiment_analysis
0,"Posted November 5, 2011.",1250,True,76561197970982479,2
1,"Posted July 15, 2011.",22200,True,76561197970982479,1
2,"Posted April 21, 2011.",43110,True,76561197970982479,2
3,"Posted June 24, 2014.",251610,True,js41637,2
4,"Posted September 8, 2013.",227300,True,js41637,2


### Modificando la columna posted

In [27]:
# Extraemos la fecha de la columna "posted" 
reviews['posted'] = reviews['posted'].str.extract(r'Posted ([\w\s\d,]+)')
reviews.head(3)

,posted,item_id,recommend,user_id,sentiment_analysis
0,"November 5, 2011",1250,True,76561197970982479,2
1,"July 15, 2011",22200,True,76561197970982479,1
2,"April 21, 2011",43110,True,76561197970982479,2


In [28]:
#Transformamos la columna 'posted' al tipo de variable datetime
reviews['posted'] = pd.to_datetime(reviews['posted'], errors='coerce')
reviews.head(3)

,posted,item_id,recommend,user_id,sentiment_analysis
0,2011-11-05,1250,True,76561197970982479,2
1,2011-07-15,22200,True,76561197970982479,1
2,2011-04-21,43110,True,76561197970982479,2


In [29]:
reviews.info() #se nos generó nuevos valores nulos

<class 'pandas.core.frame.DataFrame'>
Index: 58401 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   posted              48471 non-null  datetime64[ns]
 1   item_id             58401 non-null  int64         
 2   recommend           58401 non-null  bool          
 3   user_id             58401 non-null  object        
 4   sentiment_analysis  58401 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int64(2), object(1)
memory usage: 2.3+ MB


In [30]:
#Eliminamos valores nulos de la columna 'posted' - alrededor de 9000 filas
reviews = reviews.dropna(subset=['posted'])

In [31]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48471 entries, 0 to 59276
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   posted              48471 non-null  datetime64[ns]
 1   item_id             48471 non-null  int64         
 2   recommend           48471 non-null  bool          
 3   user_id             48471 non-null  object        
 4   sentiment_analysis  48471 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int64(2), object(1)
memory usage: 1.9+ MB


Reseteamos la numeración de los índices

In [32]:
reviews.reset_index(inplace=True)# crea nuevos índices y coloca una columna la anterior 
reviews

,index,posted,item_id,recommend,user_id,sentiment_analysis
0,0,2011-11-05,1250,True,76561197970982479,2
1,1,2011-07-15,22200,True,76561197970982479,1
2,2,2011-04-21,43110,True,76561197970982479,2
3,3,2014-06-24,251610,True,js41637,2
4,4,2013-09-08,227300,True,js41637,2
...,...,...,...,...,...,...
48466,59252,2015-10-14,730,True,wayfeng,1
48467,59255,2015-10-10,253980,True,76561198251004808,2
48468,59265,2015-10-31,730,True,72947282842,1
48469,59267,2015-12-14,730,True,ApxLGhost,2


In [33]:
#eliminamos la columna index
reviews = reviews.drop('index', axis=1)
reviews.head(5)

,posted,item_id,recommend,user_id,sentiment_analysis
0,2011-11-05,1250,True,76561197970982479,2
1,2011-07-15,22200,True,76561197970982479,1
2,2011-04-21,43110,True,76561197970982479,2
3,2014-06-24,251610,True,js41637,2
4,2013-09-08,227300,True,js41637,2


Guardamos el ETL en archivos CSV y Parquet

In [35]:
tabla = pa.Table.from_pandas(reviews) 
pq.write_table(tabla,"../0-DATA/australian_user_reviewsETL.parquet") 

In [36]:
reviews.to_csv("../0-DATA/australian_user_reviewsETL.csv", index=False, encoding="utf-8") 